In [27]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
import numpy as np
import tensorboard
import optuna

In [28]:
def make_env(reset_noise_scale, forward_reward_weight, ctrl_cost_weight, healthy_reward):
    """
    Crea e restituisce l'ambiente Ant-v5 dalla libreria Gymnasium con i parametri specificati.
    """
    # Ant-v5 è l’ambiente più recente in Gymnasium.
    return gym.make("Ant-v5", 
                    reset_noise_scale=reset_noise_scale, 
                    forward_reward_weight=forward_reward_weight, 
                    ctrl_cost_weight=ctrl_cost_weight, 
                    healthy_reward=healthy_reward, 
                    render_mode='None')

In [29]:
# Hyperparameter tuning con Optuna

def objective(trial):
    # Definisci lo spazio di ricerca degli iperparametri
    reset_noise_scale = trial.suggest_float('reset_noise_scale', 0.0, 1.0)
    forward_reward_weight = trial.suggest_float('forward_reward_weight', 0.0, 1.0)
    ctrl_cost_weight = trial.suggest_float('ctrl_cost_weight', 0.0, 1.0)
    healthy_reward = trial.suggest_float('healthy_reward', 0.0, 1.0)
    
    # Crea l'ambiente con gli iperparametri suggeriti
    env = make_env(reset_noise_scale, forward_reward_weight, ctrl_cost_weight, healthy_reward)
    env = DummyVecEnv([lambda: env])
    
    # Definisci altri iperparametri del modello
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)
    n_steps = trial.suggest_int('n_steps', 2048, 8192, step=2048)
    batch_size = trial.suggest_categorical('batch_size', [64, 128, 256])
    gamma = trial.suggest_float('gamma', 0.9, 0.9999)
    
    # Crea il modello PPO
    model = PPO("MlpPolicy", env, learning_rate=learning_rate, n_steps=n_steps, batch_size=batch_size, gamma=gamma, verbose=0)
    
    # Addestra il modello
    model.learn(total_timesteps=100000)
    
    # Valuta il modello
    episodes = 50
    episode_rewards = []
    for episode in range(episodes):
        obs = env.reset()
        done = False
        episode_reward = 0
        while not done:
            action, _states = model.predict(obs)
            obs, reward, done, info = env.step(action)
            episode_reward += reward
        episode_rewards.append(episode_reward)
    
    # Ritorna la ricompensa media come obiettivo
    return np.mean(episode_rewards)

# Crea uno studio Optuna e ottimizza l'obiettivo
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Stampa i migliori iperparametri trovati
print("Best hyperparameters: ", study.best_params)

[I 2025-02-09 17:08:32,929] A new study created in memory with name: no-name-8a58bc06-c13f-431d-8d47-1ed6676af416
/var/folders/5w/qb_kxxjs5lscg9_8tpttrzs40000gn/T/ipykernel_92434/3712145730.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)
[I 2025-02-09 17:09:22,478] Trial 0 finished with value: -1.9567195177078247 and parameters: {'reset_noise_scale': 0.9720820487461717, 'forward_reward_weight': 0.6844753225967173, 'ctrl_cost_weight': 0.2775315872821543, 'healthy_reward': 0.6787322944822765, 'learning_rate': 2.9682299178353676e-05, 'n_steps': 4096, 'batch_size': 64, 'gamma': 0.9403652492260215}. Best is trial 0 with value: -1.9567195177078247.
[I 2025-02-09 17:10:05,771] Trial 1 finished with value: 11.31909465789795 and parameters: {'reset_noise_scale': 

Best hyperparameters:  {'reset_noise_scale': 0.2282706739101626, 'forward_reward_weight': 0.09314040045482441, 'ctrl_cost_weight': 0.028140178122103423, 'healthy_reward': 0.9926479631637423, 'learning_rate': 0.0008676828845312949, 'n_steps': 4096, 'batch_size': 64, 'gamma': 0.9328230070576791}
